In [1]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.amazon.amazon_estimator import get_image_uri

In [12]:
bucket='sagemaker-builtin-imageclass-cancer-detection'
input_prefix='breast-cancer-detection/input/recordio'
input_train='s3://{}/{}/train'.format(bucket,input_prefix)
input_test='s3://{}/{}/test'.format(bucket,input_prefix)

In [3]:
output_prefix='breast-cancer-detection/output'
output_path='s3://{}/{}/'.format(bucket,output_prefix)

In [4]:
# Cofiguring the hyperparameters

num_classes=2
num_trainnig_samples=194266
num_layers=18
mini_batch_size=128
image_shape=(3,50,50)
augmentation_type='crop_color_transform'
epochs=5
learning_rate=0.01
use_pretrained_model=1



In [5]:
instance_count=1
instance_type='ml.p2.16xlarge'
volume_size_gb=50

In [6]:
role=get_execution_role()
training_image=get_image_uri(boto3.Session().region_name,'image-classification')

'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


In [7]:
train_timeout=360000


In [8]:
session=sagemaker.Session()


In [9]:
estimator=sagemaker.estimator.Estimator(training_image,
                                       role,
                                       train_instance_count=instance_count,
                                       train_instance_type=instance_type,
                                       train_volume_size=volume_size_gb,
                                       train_max_run=train_timeout,
                                       output_path=output_path,
                                       base_job_name='sm-builtin-img-classification'
                                       )

Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.


In [10]:
estimator.set_hyperparameters(num_classes=num_classes,
                             num_trainnig_samples=num_trainnig_samples,
                             num_layers=num_layers,
                             mini_batch_size=mini_batch_size,
                             image_shape=image_shape,
                             augmentation_type=augmentation_type,
                             epochs=epochs,
                             learning_rate=learning_rate,
                             use_pretrained_model=use_pretrained_model)

In [13]:
estimator.fit(inputs=input_train)

's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.


2020-10-06 09:31:04 Starting - Starting the training job...
2020-10-06 09:31:06 Starting - Launching requested ML instances............
2020-10-06 09:33:23 Starting - Preparing the instances for training............
2020-10-06 09:35:20 Downloading - Downloading input data...
2020-10-06 09:35:42 Training - Downloading the training image...
2020-10-06 09:36:32 Uploading - Uploading generated training modelDocker entrypoint called with argument(s): train
[10/06/2020 09:36:30 INFO 140575959942976] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/image_classification/default-input.json: {u'beta_1': 0.9, u'gamma': 0.9, u'beta_2': 0.999, u'optimizer': u'sgd', u'use_pretrained_model': 0, u'eps': 1e-08, u'epochs': 30, u'lr_scheduler_factor': 0.1, u'num_layers': 152, u'image_shape': u'3,224,224', u'precision_dtype': u'float32', u'mini_batch_size': 32, u'weight_decay': 0.0001, u'learning_rate': 0.1, u'momentum': 0}
[10/06/2020 09:36:30 INFO 140575959942976] Merging with 

UnexpectedStatusException: Error for Training job sm-builtin-img-classification-2020-10-06-09-31-04-084: Failed. Reason: ClientError: No value(s) were specified for 'num_training_samples' which are required hyperparameter(s) (caused by ValidationError)

Caused by: u'num_training_samples' is a required property

Failed validating u'required' in schema:
    {u'$schema': u'http://json-schema.org/schema#',
     u'additionalProperties': False,
     u'definitions': {u'boolean_0_1': {u'oneOf': [{u'enum': [u'0',
                                                             u'1'],
                                                   u'type': u'string'},
                                                  {u'enum': [0, 1],
                                                   u'type': u'number'}]},
                      u'boolean_true_false_0_1': {u'oneOf': [{u'enum': [u'true',
                                                                        u'false',
                                             